In [176]:
import matplotlib.pyplot as plt
import numpy as np
import pandas

MTOW = 15
BATT_WEIGHT = 3

# CONSTANTS
AIR_DEN = 1.25
GRAVITY = 9.82

MAGIC_NUMBER = 0.6171062

prop1 = np.genfromtxt("0.291667.csv", delimiter=",")
prop2 = np.genfromtxt("0.416667.csv", delimiter=",")
prop3 = np.genfromtxt("0.625.csv", delimiter=",")
prop4 = np.genfromtxt("1.csv", delimiter=",")

prop_pd = np.array([0.21667, 0.416667, 0.625, 1])

prop_data = [
    prop1,
    prop2,
    prop3,
    prop4
]


def calculate_score(takeoff_dist,
                    max_speed, 
                    min_speed, 
                    flight_distance,
                    battery_capacity,
                    payload_mass,
                    payload_volume,
                    landing_angle=5, 
                    landing_dist=10,
                    climb_angle=20, 
                    glide_ratio=3, ):

    payload_energy_consumed = (battery_capacity/(4.2*6))/(flight_distance*payload_mass)

    reference_airplane = np.array([
        30,		# Take-off Distance
        12,		# Climb Angle
        3,		# Glide Ratio
        30,		# Max Speed
        10,		# Min Speed
        5,		# Landing Angle
        50,		# Landing Distance
        15,		# Flight Distance
        0.4,	# Specific Energy Consumtion
        5,		# Payload Mass
        0.5		# Payload Volume
	])
    
    our_plane = np.array([
        takeoff_dist,		# Take-off Distance
        climb_angle,		# Climb Angle
        glide_ratio,		# Glide Ratio
        max_speed,		# Max Speed
        min_speed,		# Min Speed
        landing_angle,		# Landing Angle
        landing_dist,		# Landing Distance
        flight_distance,		# Flight Distance
        payload_energy_consumed,	# Specific Energy Consumtion
    	payload_mass,		# Payload Mass
        payload_volume		# Payload Volume
	])
    
    weights = np.array([
        -4,
        7,
        25,
        4,
        -4,
        5,
        -3,
        8,
        -12,
        10,
        18
	])
    div = our_plane/reference_airplane
    scores = weights*np.tanh(div - 1)
    score = np.sum(scores)

    #print("Div:", div)
    #print("Scores: ", scores)
    
    print("Take off Distance: ", scores[0])
    print("Climb angle: ", scores[1])
    print("Glide Ratio ", scores[2])
    print("Max Speed: ", scores[3])
    print("Min Speed: ", scores[4])
    print("Landing Angle: ", scores[5])
    print("Landing Distance: ", scores[6])
    print("Flight Distanace: ", scores[7])
    print("Specific Energy Consumtion: ", scores[8])
    print("Payload Mass: ", scores[9])
    print("Payload Volume: ", scores[10])

    print(score)
    


In [165]:
def prop_pt_calc(prop_matrix, airspeed, diameter):
    max_rpm = 60/(np.pi*(diameter)*0.00291545)

    m, b = np.polyfit([0, 1], [100, max_rpm], 1)
    peak_eff_rpm = MAGIC_NUMBER*m+b

    take_off_advance_ratio = airspeed/(peak_eff_rpm/60*diameter)
    idx = (np.abs(prop_matrix[:, 0] - take_off_advance_ratio)).argmin()

    coeff_thrust = prop_matrix[idx, 1]
    coeff_power = prop_matrix[idx, 2]


    thrust = coeff_thrust*AIR_DEN*np.power(peak_eff_rpm/60, 2)*np.power(diameter, 4)
    power = coeff_power*AIR_DEN*np.power(peak_eff_rpm/60, 3)*np.power(diameter, 5)

    return power, thrust

In [166]:
def take_off(prop_matrix, prop_diameter, wing_area, fusilage_area, take_off_cd, take_off_cl):
    stall_velocity = (2*MTOW*GRAVITY/(take_off_cl*AIR_DEN*wing_area))**0.5

    #takeoff_drag = 0.5*takeoff_cd*AIR_DEN*(fusilage_area + wing_area)*takeoff_velocity**2
    #takeoff_accel = (max_thrust - takeoff_drag)/MTOW

    # hard to calculate power and thrust without minimum 5 m/s
    power, thrust = prop_pt_calc(prop_matrix=prop_matrix, airspeed=5, diameter=prop_diameter)

    dt = 0.0001
    current_velocity = 0
    current_time = 0
    current_energy = 0
    current_distance = 0

    takeoff_accel = np.array(())
    takeoff_velocity = np.array(())
    x = np.array(())



    while current_velocity < stall_velocity:
        current_drag = 0.5*take_off_cd*AIR_DEN*(fusilage_area + wing_area)*current_velocity**2
        current_accel = (thrust - current_drag)/MTOW
        
        current_distance = current_distance + current_velocity*dt + 0.5*current_accel*dt**2
        current_velocity = current_velocity + current_accel*dt
        
        current_energy = current_energy + power*dt
        
        current_time = current_time+dt
        
        #takeoff_accel = np.append(takeoff_accel, current_accel)
        #takeoff_velocity = np.append(takeoff_velocity, current_velocity)
        #x = np.append(x, current_time)
        
    takeoff_time = current_time
    takeoff_energy = current_energy/3600
    takeoff_distance = current_distance

    print(f"Takeoff time: {round(takeoff_time,1)} s")
    print(f"Energy: {round(takeoff_energy,1)} Wh")
    print(f"Takeoff distance: {round(takeoff_distance, 1)} m")

    return takeoff_distance, takeoff_energy, stall_velocity

In [103]:
def climb():
    drag = 0.5*cruise_cd*AIR_DEN*(fusilage_area + wing_area)*cruise_airspeed**2
    lift = 0.5*cruise_cl*AIR_DEN*(wing_area)*cruise_airspeed**2

In [184]:
def power_from_drag(prop_matrix, airspeed, diameter, drag):
    max_rpm = 60/(np.pi*(diameter)*0.00291545)

    m, b = np.polyfit([0, 1], [100, max_rpm], 1)
    peak_eff_rpm = MAGIC_NUMBER*m+b
    print(max_rpm)
    print(peak_eff_rpm)

    advance_ratio = airspeed/(peak_eff_rpm/60*diameter)
    idx = (np.abs(prop_matrix[:, 0] - advance_ratio)).argmin()

    coeff_thrust = prop_matrix[idx, 1]
    coeff_power = prop_matrix[idx, 2]

    rps = np.power(drag/(coeff_thrust*AIR_DEN*np.power(diameter, 4)), 1/2)
    power = coeff_power*AIR_DEN*np.power(rps, 3)*np.power(diameter, 5)

    return power

In [188]:
def endurance(prop_matrix, prop_diameter, cruise_speed, energy_left, wing_area, fusilage_area, cruise_cd):
    area_drag = wing_area + fusilage_area
    cruise_drag = 0.5*AIR_DEN*cruise_cd*area_drag*cruise_speed**2+1

    power = power_from_drag(prop_matrix=prop_matrix, airspeed=cruise_speed, diameter=prop_diameter, drag=cruise_drag)

    cruise_time = energy_left/power

    # Assuming airspeed = ground speed
    cruise_distance = cruise_speed/60*cruise_time
    print("cruise power: ", power)
    print("cruise drag", cruise_drag)
    print(f"Cruise Time: {round(cruise_time, 1)}")
    print(f"Cruise Distance: {round(cruise_distance, 1)}")
    return cruise_distance


In [ ]:
def generate_prop_matrix(diameter, pitch):
    pd = pitch/diameter

    diameter = diameter*25.4
    print(pd)
    pd_upperbound = 0
    pd_lowerbound = 0

    if pd >= prop_pd.max() or pd < prop_pd.min():
        print("Incorrect PD value")
        return False

    for idx, i in enumerate(prop_pd):
        if pd < i:
            pd_upperbound = i
            #print(idx_lowerbound)
            break

    for i in reversed(prop_pd):
        if pd > i:
            pd_upperbound = i
            break

    print(pd_upperbound)
    print(pd_lowerbound)
    print(idx)
    m, b = np.polyfit([pd_lowerbound, pd_upperbound], [0, 1], 1)
    
    moj = m*pd+b
    print(moj)
    new_matrix = (1-moj)*prop_data[idx-1] + moj*prop_data[idx]
    return new_matrix



0.9545454545454546
0.625
0
3
1.527272727272727


In [189]:
# Energy storage in Wh, assuming 260 Wh/kg and 3 kg of batteries
battery_energy = 360*3

landing_gear_weight = 0.95
motor_weight = 0.15
prop_weight = 0.05

main_body = 5
fligtcontroller = 0.07

payload = 15 - BATT_WEIGHT - landing_gear_weight - motor_weight*2 - prop_weight*2 - main_body - fligtcontroller
payload_vol = 0.5
# Needs a better estimation of Cd_fuse!!! Scaled with fuselage surface area
fusilage_cd = 0.02

#----- Optim
cruise_airspeed = 15 # m/s
wing_area = 2 # m^2
fusilage_area = 1 # m^2

diameter = 16 # inches
pitch = 8 # inches
p_d = diameter/pitch
prop_matrix = generate_prop_matrix(diameter, pitch)
diameter = diameter*25.4/1000 # meters

max_rpm = 60/(np.pi*(diameter)*0.00291545)

m, b = np.polyfit([0, 1], [100, max_rpm], 1)
peak_eff_rpm = MAGIC_NUMBER*m+b
print(max_rpm)
print(peak_eff_rpm)


take_off_cd = 0.13
take_off_cl = 1.3

cruise_cd = 0.01

take_off_cd = fusilage_cd + take_off_cd

take_off_dist, take_off_energy, stall_velocity = take_off(
    prop_matrix=prop_matrix,
    prop_diameter=diameter,
    wing_area=wing_area,
    fusilage_area=fusilage_area,
    take_off_cd=take_off_cd,
    take_off_cl=take_off_cl
    )

endurance_energy = battery_energy - take_off_energy




cruise_distance = endurance(
    prop_matrix=prop_matrix,
    prop_diameter=diameter,
    cruise_speed=cruise_airspeed,
    energy_left=endurance_energy, 
    wing_area=wing_area, 
    fusilage_area=fusilage_area, 
    cruise_cd=cruise_cd, 
)
print("=====================")
print("SCORES")

calculate_score(takeoff_dist=take_off_dist,
                max_speed=30,
                min_speed=stall_velocity,
                flight_distance=cruise_distance,
                battery_capacity=battery_energy,
                payload_mass=payload,
                payload_volume=payload_vol)

0.5
0.416667
0
2
1.1999990400007676
16119.147922481612
9985.51550168052
Takeoff time: 1.7 s
Energy: 1.1 Wh
Takeoff distance: 8.6 m
16119.147922481612
9985.51550168052
cruise power:  37.64121883497716
cruise drag 5.218750000000001
Cruise Time: 28.7
Cruise Distance: 7.2
SCORES
Take off Distance:  2.449922592780311
Climb angle:  4.079480617435371
Glide Ratio  0.0
Max Speed:  0.0
Min Speed:  0.1915224910222708
Landing Angle:  0.0
Landing Distance:  1.9921103108035472
Flight Distanace:  -3.835619333945679
Specific Energy Consumtion:  -11.193600244669936
Payload Mass:  1.1548248661898017
Payload Volume:  0.0
-5.161358700384314
